#### Step2: Document Agent Processing

##### 1. Load documents

In [2]:
from config import RESOURCES_PATH
from document_agent import DocumentAgentSop
from document_loader import load_sop_documents
from extract_metadata import extract_metadata_from_docx
from text_processing import dynamic_text_splitter,get_or_cache_embeddings
from vector_db import setup_chroma
from retriever import setup_sop_retriever
from reranker import rerank_documents
from llm import get_llm
from config import RESOURCES_PATH
from langchain_ollama import OllamaEmbeddings


C:\Users\mnmhy\PycharmProjects\agenticrag\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mnmhy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mnmhy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loaded workinst embeddings from cache.


In [3]:

sop_docs = load_sop_documents(RESOURCES_PATH)
print("[DEBUG] Loaded Work Instructions:", sop_docs[0].page_content[:5000])


[DEBUG] extract_metadata_from_docx() function started.
[DEBUG] Found 6 tables in the document.
[DEBUG] Processing Table 1

[DEBUG] Table 1 Extracted Text:
AG SOLUTION MANUFACTURING PROCEDURE Review: 9 Date: 08/02/2021 | CONFIDENTIAL PRODUCT: ALKYLBENZEN SULFONIC ACID (11027563)

[MATCH] Found Review: 9
[MATCH] Found Date: 08/02/2021
[DEBUG] Processing Table 2

[DEBUG] Table 2 Extracted Text:
PLANT : | SULFAX |  |  | 
REACTOR : | R503 |  |  | 

[DEBUG] Processing Table 3

[DEBUG] Table 3 Extracted Text:
HAZARD CLASSIFICATION OF THE FINAL PRODUCT: | HAZARD CLASSIFICATION OF THE FINAL PRODUCT: | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage. | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage. | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage.
MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE
SAP CODE | DESCRIPTION

#### Debugging why metadata is not extracted
##### one issues was even it can extract all text from docs (even table content), it cannot extract metadata like review and date. The reason why it happens was
##### - Multi-line table cells : Date and Review are inside the same cell as other text nad regex search misses multi-ine text

In [3]:
from docx import Document

file_path = r"C:\Users\mnmhy\PycharmProjects\agenticrag\Resources\Alkylbenzen Sulfonic Acid.docx"

def debug_docx_tables(file_path):
    doc = Document(file_path)

    print("\n[DEBUG] Checking if document contains tables...")
    if not doc.tables:
        print("[ERROR] No tables found in the document!")
        return

    print(f"[DEBUG] Found {len(doc.tables)} tables in the document.")

    for table_idx, table in enumerate(doc.tables):
        print(f"\n--- Table {table_idx + 1} ---")
        for row in table.rows:
            for cell in row.cells:
                print(f"  [CELL] {cell.text.strip()}")

debug_docx_tables(file_path)


[DEBUG] Checking if document contains tables...
[DEBUG] Found 6 tables in the document.

--- Table 1 ---
  [CELL] AG SOLUTION
MANUFACTURING PROCEDURE
Review: 9
Date: 08/02/2021
  [CELL] CONFIDENTIAL
PRODUCT: ALKYLBENZEN SULFONIC ACID (11027563)

--- Table 2 ---
  [CELL] PLANT :
  [CELL] SULFAX
  [CELL] 
  [CELL] 
  [CELL] 
  [CELL] REACTOR :
  [CELL] R503
  [CELL] 
  [CELL] 
  [CELL] 

--- Table 3 ---
  [CELL] HAZARD CLASSIFICATION OF THE FINAL PRODUCT:
  [CELL] HAZARD CLASSIFICATION OF THE FINAL PRODUCT:
  [CELL] H302- Harmful if swallowed.
H314-Causes severe skin burns and serious eye damage.
  [CELL] H302- Harmful if swallowed.
H314-Causes severe skin burns and serious eye damage.
  [CELL] H302- Harmful if swallowed.
H314-Causes severe skin burns and serious eye damage.
  [CELL] MANUFACTURING PROCEDURE
  [CELL] MANUFACTURING PROCEDURE
  [CELL] MANUFACTURING PROCEDURE
  [CELL] MANUFACTURING PROCEDURE
  [CELL] MANUFACTURING PROCEDURE
  [CELL] SAP CODE
  [CELL] DESCRIPTION
  [CELL] DE

In [4]:
file_path = r"C:\Users\mnmhy\PycharmProjects\agenticrag\Resources\Alkylbenzen Sulfonic Acid.docx"
metadata = extract_metadata_from_docx(file_path)
print(metadata)

[DEBUG] extract_metadata_from_docx() function started.
[DEBUG] Found 6 tables in the document.
[DEBUG] Processing Table 1

[DEBUG] Table 1 Extracted Text:
AG SOLUTION MANUFACTURING PROCEDURE Review: 9 Date: 08/02/2021 | CONFIDENTIAL PRODUCT: ALKYLBENZEN SULFONIC ACID (11027563)

[MATCH] Found Review: 9
[MATCH] Found Date: 08/02/2021
[DEBUG] Processing Table 2

[DEBUG] Table 2 Extracted Text:
PLANT : | SULFAX |  |  | 
REACTOR : | R503 |  |  | 

[DEBUG] Processing Table 3

[DEBUG] Table 3 Extracted Text:
HAZARD CLASSIFICATION OF THE FINAL PRODUCT: | HAZARD CLASSIFICATION OF THE FINAL PRODUCT: | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage. | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage. | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage.
MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE
SAP CODE | DESCRIPTION

##### 2. Document chunking

###### Metadata is not correctly extracted as the output shows Metadata: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': None, 'Date': None} and it was causing Value Error in next step

In [5]:

sop_chunks = dynamic_text_splitter(sop_docs, default_chunk_size=800)

for i, chunk in enumerate(sop_chunks[:5]):
    print(f"\n===Chunk {i+1}===")
    print(f"Metadata: {chunk.metadata}")
    print(f"Content (first 500 chars): {chunk.page_content[:1000]}\n")



===Chunk 1===
Metadata: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
Content (first 500 chars): HAZARDOUS OPERATIONS & PREVENTIVE MEASURES
Always use standard protective gear (workwear, helmet, safety boots, protective glasses, and face shield when handling any product) and standard chemical protection gloves. If you come into contact with any product, wash immediately and seek medical attention. TABLE DATA:


===Chunk 2===
Metadata: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
Content (first 500 chars): HAZARDOUS OPERATIONS & PREVENTIVE MEASURES
Always use standard protective gear (workwear, helmet, safety boots, protective glasses, and face shield when handling any product) and standard chemical protection gloves. If you come into contact with any product, wash immediately and seek medical attention. TABLE DATA: MANUFACTURING PROCEDURE
Review: 9
Date: 08/02/2021 | CONFIDENTIAL
PRODUCT: ALKYLBENZEN SULFONIC

##### 3. Embedding Generation

In [6]:
embedding_model = OllamaEmbeddings(model="mxbai-embed-large")

embeddings = get_or_cache_embeddings(sop_chunks, "sop", embedding_model)

print(f"Example embedding vector (first 5 values): {embeddings[0][:5]}")
print(f"total embeddings generated: {len(embeddings)}")


Generating sop embeddings...


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


sop embeddings cached.
Example embedding vector (first 5 values): [-0.017540157, 0.03415288, 0.0031194747, 0.02768372, -0.011931023]
total embeddings generated: 190


##### 4. Vector Database Storage

In [7]:
from retriever import filter_complex_metadata

for doc in sop_chunks:
    print("[DEBUG] Metadata before filtering:", doc.metadata)  # Check original metadata
    filtered_metadata = filter_complex_metadata(doc.metadata)
    print("[DEBUG] Metadata after filtering:", filtered_metadata)

vector_store = setup_chroma(sop_chunks)
print("Vector database successfully set up!")

[DEBUG] Metadata before filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata after filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata before filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata after filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata before filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata after filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata before filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata after filtering: {'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9', 'Date': '08/02/2021'}
[DEBUG] Metadata before filtering: {'Document Name':

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


[DEBUG] Adding 190 SOP documents to ChromaDB.


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Vector database successfully set up!


##### 5. Retrieval & Reranking

In [8]:

doc_agent = DocumentAgentSop(debug_mode=True)
llm_instance = doc_agent.llm_instance
retriever = setup_sop_retriever(sop_chunks)
query = "what are the raw material needed to produce Alkylbenzen Sulfonic Acid?"
retrieved_docs = retriever.invoke({"query": query})

def trace_reranking(query, retrieved_docs):
    if "source_documents" in retrieved_docs and retrieved_docs["source_documents"]:
        document_dicts = [{"page_content": doc.page_content, "metadata": doc.metadata}
                          for doc in retrieved_docs["source_documents"]]
        reranked_docs = rerank_documents(query, document_dicts, llm_instance)
        for i, doc in enumerate(reranked_docs[:3]):
            print(f"\n=== Reranked Document {i+1} ===")
            print(f"Metadata: {doc.get('metadata', {})}")
            print(f"Content (first 500 chars): {doc['page_content'][:500]}\n")
    else:
        print("No relevant documents retrieved for reranking.")

trace_reranking(query, retrieved_docs)

C:\Users\mnmhy\PycharmProjects\agenticrag\llm.py:80: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  self._ollama = Ollama(model=model, temperature=temperature)


[DEBUG] extract_metadata_from_docx() function started.
[DEBUG] Found 6 tables in the document.
[DEBUG] Processing Table 1

[DEBUG] Table 1 Extracted Text:
AG SOLUTION MANUFACTURING PROCEDURE Review: 9 Date: 08/02/2021 | CONFIDENTIAL PRODUCT: ALKYLBENZEN SULFONIC ACID (11027563)

[MATCH] Found Review: 9
[MATCH] Found Date: 08/02/2021
[DEBUG] Processing Table 2

[DEBUG] Table 2 Extracted Text:
PLANT : | SULFAX |  |  | 
REACTOR : | R503 |  |  | 

[DEBUG] Processing Table 3

[DEBUG] Table 3 Extracted Text:
HAZARD CLASSIFICATION OF THE FINAL PRODUCT: | HAZARD CLASSIFICATION OF THE FINAL PRODUCT: | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage. | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage. | H302- Harmful if swallowed. H314-Causes severe skin burns and serious eye damage.
MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE | MANUFACTURING PROCEDURE
SAP CODE | DESCRIPTION

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:azure.identity._credentials.environment:Incomplete environment configuration for EnvironmentCredential. These variables are set: AZURE_TENANT_ID
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.7 (Windows-11-10.0.26100-SP0)'
No body was attached to the request


Loaded sop embeddings from cache.


INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/b98e9951-860f-464a-a9a2-f69802ca8721/resourceGroups/ai_llm/providers/Microsoft.MachineLearningServices/workspaces/agentic_rag/connections?api-version=REDACTED&category=REDACTED&includeAll=REDACTED'
Request method: 'GET'
Request headers:
    'Accept': 'application/json'
    'x-ms-client-request-id': '05549ac6-023a-11f0-806b-f8b54d499b92'
    'User-Agent': 'azsdk-python-ai-projects/1.0.0b6 Python/3.12.7 (Windows-11-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Cache-Control': 'no-cache'
    'Pragma': 'no-cache'
    'Content-Length': '3387'
    'Content-Type': 'application/json; charset=utf-8'
    'Expires': '-1'
    'Vary': 'REDACTED'
    'x

[DEBUG] Adding 190 SOP documents to ChromaDB.


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://kdgthelab.cognitiveservices.azure.com/models/chat/completions?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '2971'
    'Accept': 'application/json'
    'x-ms-client-request-id': '47933a63-023a-11f0-a38d-f8b54d499b92'
    'api-key': 'REDACTED'
    'User-Agent': 'AIProjectClient azsdk-python-ai-inference/1.0.0b9 Python/3.12.7 (Windows-11-10.0.26100-SP0)'
    'Authorization': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Length': '974'
    'Content-Type': 'application/json'
    'azureml-internal-engineid': 'REDACTED'
    'azureml-model-id': 'REDACTED'
    'openai-internal-encodi


=== Reranked Document 1 ===
Metadata: {'Date': '08/02/2021', 'Document Name': 'Alkylbenzen Sulfonic Acid', 'Review': '9'}
Content (first 500 chars): Start circulating dodecylbenzene through the reactor at the desired flow rate and activate cooling.'}, {'Raw Material Loading': '7. Begin sulfonation with the following conditions:'}, {'Raw Material Loading': '8. Modify gas splitting conditions if required'}, {'Raw Material Loading': 'When there is a visible chance in the ester acid drop in the reactor, send the output from the tank into the maturing device.'}, {'Raw Material Loading': 'Check acidic index and ester acid colour.

